In [7]:
#Transient simulation of diffusio-osmosis flows through a narrow channel 
#From P. Bacchin, University of Toulouse, France 
#Equations are solved with the fipy platform from http://www.ctms.nist.gov/fipy
#You are free to use this code but :
#If you use this code for your research, please cite : 
# https://dx.doi.org/10.1016/j.ces.2016.10.024 paper that establishes the energy map model for colloids or 
# https://doi.org/10.1016/j.colsurfa.2017.08.034 paper that applies the model to simulate osmosis in 1D and
# Interfacially driven transport in narrow channel. Journal of Physics: Condensed matter. 2018 (for the 2D simulations)
#If you are improving significantly this code, please send an updated copy to patrice.bacchin@univ-tlse3.fr
#Enjoy diffusio-osmotic flows dynamics !
from pylab import *
import sys
from fipy import *
from fipy.tools import numerix
from fipy.variables.faceGradVariable import _FaceGradVariable
from mpl_toolkits.mplot3d import Axes3D

#DATA
#Flow
viscosity = 5.55555555556e-06 
Pe =0.
Pey=0.
#Colloid concentration
phi_init=0.
phi_c=0.01
#Energy map water-interface equivalent to a penalisation method
pfi=1.
lfi=0.01
#Energy map interface-colloid (value at 10-5 M)
xic= 0.
kic= 100.
lic= 0.1
hic= 0.

#SOLVING PARAMETERS
#time steps 
dt=1.e-2
steps=101
#tolerance for momemtum and mass balance
tol=1.e-5
resu_tol= tol
resphi_tol = tol
pres_tol=tol
continuity_tol=tol
#mesh
Ly=2.
Nx =200
Ny=100
Lx=Ly*Nx/Ny
dL=Ly/Ny
mesh = PeriodicGrid2DTopBottom(nx=Nx, ny=Ny, dx=dL, dy=dL)
X, Y = mesh.faceCenters
x, y = mesh.cellCenters
pressureRelaxation = 0.8
velocityRelaxation = 0.5

#VARIABLES
#momentum
pressure = CellVariable(mesh=mesh, hasOld=True, name='Pressure')
pressureCorrection = CellVariable(mesh=mesh, hasOld=True)
xVelocity = CellVariable(mesh=mesh, hasOld=True,  name='Pe X')
yVelocity = CellVariable(mesh=mesh,hasOld=True,  name='Pe Y')
velocity = FaceVariable(mesh=mesh, rank=1)
#mass
phi = CellVariable(mesh=mesh, hasOld=True, value=phi_init,name='Volume fraction')

#BOUNDARY CONDITIONS ON THE DOMAIN + PERIODIC TOP&BOTTOM (no conditions -> gradient of zero)
#momentum
xVelocity.constrain(Pe, mesh.facesRight )
yVelocity.constrain(Pey, mesh.facesRight)
pressureCorrection.constrain(0., mesh.facesLeft & (Y < dL))
#mass
phi.constrain(phi_c, mesh.facesLeft)
phi.faceGrad.constrain(0., mesh.facesRight)

#SOLID CHANNEL GEOMETRY
xup, xdw, xch=3.,4.,3.
yup, ych=0.,0.5
#level set function to define the distance to the wall
var1 = DistanceVariable(name='level set variable',mesh=mesh,value=-1.)
var2 = DistanceVariable(name='level set variable',mesh=mesh,value=-1.)
var1.setValue(1., where=(x >= xup) & (x <= xdw) & (y <= ych) & (x >= xup+(y-yup)*(xch-xup)/(ych-yup)))
var2.setValue(1., where=(x >= xup) & (x <= xdw) & (y >= (Ly-ych)) & (x >= xup+(y-(Ly-yup))*(xch-xup)/(-ych+yup)))
var1.calcDistanceFunction(order=1)
var2.calcDistanceFunction(order=1)
#Building the colloid-interface energy map
pi_ci = CellVariable(mesh=mesh, value=0.,name='Colloid-interface energy map')
pi_ci.setValue(kic*exp(var1/lic)+kic*exp(var2/lic), where=(var1 < 0) & (var2 < 0))
pi_ci.setValue(kic, where=(var1 > 0))
pi_ci.setValue(kic, where=(var2 > 0))
#Building the fluid-interface energy map
pi_fi= CellVariable(mesh=mesh, value=0.,name='Fluid-interface energy map')
pi_fi.setValue(pfi, where=(var1 > 0))
pi_fi.setValue(pfi, where=(var2 > 0))
pi_fi.setValue(pfi*exp(var1/lfi)+pfi*exp(var2/lfi), where=(var1 < 0) & (var2 < 0) )

#EQUATIONS                                                                                                                                                                            
#momentum equation (faut-il prendre le grad des contributions des deux parois ?)
xVelocityEq = DiffusionTerm(coeff=viscosity) - pressure.grad.dot([1.,0.]) - ImplicitSourceTerm(pi_fi) - (phi)*(pi_ci.grad.dot([1.,0.]))
yVelocityEq = DiffusionTerm(coeff=viscosity) - pressure.grad.dot([0.,1.]) - ImplicitSourceTerm(pi_fi) - (phi)*(pi_ci.grad.dot([0.,1.]))
#mass balance equation
eq0 = TransientTerm() + ConvectionTerm(velocity-pi_ci.faceGrad) == DiffusionTerm(coeff=1)
#corrections to satisfy the continuity equations
ap = CellVariable(mesh=mesh, value=1.)
coeff = 1./ ap.arithmeticFaceValue*mesh._faceAreas * mesh._cellDistances 
pressureCorrectionEq = DiffusionTerm(coeff=coeff) - velocity.divergence
volume = CellVariable(mesh=mesh, value=mesh.cellVolumes, name='Volume')
contrvolume=volume.arithmeticFaceValue

for step in range(steps): 
    sweep=0.
    xres=1000.
    pres=1000.
    cont=1000.
    pressure.updateOld()
    pressureCorrection.updateOld()
    xVelocity.updateOld()
    yVelocity.updateOld()
    while (xres > resu_tol or pres > pres_tol or cont > continuity_tol) :
        sweep=sweep+1       
        ## solve the Stokes equations to get starred values
        xVelocityEq.cacheMatrix()
        xres = xVelocityEq.sweep(var=xVelocity,underRelaxation=velocityRelaxation)
        xmat = xVelocityEq.matrix
        yres = yVelocityEq.sweep(var=yVelocity,underRelaxation=velocityRelaxation)
        ## update the ap coefficient from the matrix diagonal
        ap[:] = -xmat.takeDiagonal()
        ## update the face velocities based on starred values with the Rhie-Chow correction.
        ## cell pressure gradient
        presgrad = pressure.grad
        ## face pressure gradient
        facepresgrad = _FaceGradVariable(pressure)
        velocity[0] = xVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[0].arithmeticFaceValue-facepresgrad[0])
        velocity[1] = yVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[1].arithmeticFaceValue-facepresgrad[1])
        #velocity[0, mesh.facesLeft.value] = Pe
        velocity[0, mesh.facesRight.value] = 0.
        #velocity[1, mesh.facesLeft.value] = Pey
        velocity[1, mesh.facesRight.value] = 0.
        ## solve the pressure correction equation
        pressureCorrectionEq.cacheRHSvector()
        ## left bottom point must remain at pressure 0, so no correction
        pres = pressureCorrectionEq.sweep(var=pressureCorrection)
        rhs = pressureCorrectionEq.RHSvector
        ## update the pressure using the corrected value
        pressure.setValue(pressure + pressureRelaxation * pressureCorrection )
        ## update the velocity using the corrected pressure
        xVelocity.setValue(xVelocity - pressureCorrection.grad[0] / ap * mesh.cellVolumes)
        yVelocity.setValue(yVelocity - pressureCorrection.grad[1] / ap * mesh.cellVolumes)
        cont=max(abs(rhs))
        if sweep % 10 == 0 :
            print ('step:', step, 'sweep:', sweep,', x residual:',xres, ', y residual',yres, ', p residual:',pres, ', continuity:',cont)
    phi.updateOld()
    res0 = 1e100
    t=step*dt
    while  res0 > resphi_tol:
         #solving      
         res0 = eq0.sweep(var=phi, dt=dt)
    if (step ==50 or step ==100 or step ==200 or step ==300 or step ==500 or step ==1000) :
         print ('step:', step, ', phi-moy:',mean(phi.value), 'continuity:',max(abs(rhs))    )    
         

C:\Users\patri\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


step: 50 , phi-moy: 0.0020095246741259325 continuity: 2.728780605145239e-06


KeyboardInterrupt: 